<a href="https://colab.research.google.com/github/BriacLEGUILLOU/Impact_score/blob/master/Impact_score_pr%C3%A9paration_des_donn%C3%A9es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd

# Load of bundle
chemin = '/content/gdrive/My Drive/Impact Score/Données partagées pour Impact Score/Données/'
bundle_raw = pd.read_csv(chemin + 'bundle.csv', sep=';', header=None,  index_col=False, names =['siren', 'year', 'accountability', 'bundle','amount'])
bundle_raw.head()

,siren,year,accountability,bundle,amount
0,5420120,2016,0,10,781843
1,5420120,2016,0,1,11836
2,5420120,2016,0,13,27644
3,5420120,2016,0,15,866430
4,5420120,2016,0,16,214907


# 1 Mise en forme des données

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
pd.set_option('display.max_rows', 60)

In [4]:
def obtention_log10(val):
  """ Permet de convertir un nombre de la façon suivante : 100000 devient 5 et -100 devient -2 """
  if val == np.nan: return np.nan
  elif (isinstance(val, float)) or (isinstance(val, int)):
    signe, nb = np.sign(val), np.absolute(val)
    if nb < 1: return 0
    else : return math.log10(nb) * signe
  else: return val

In [5]:
# Identification des doublons
# bundle_raw[bundle_raw.duplicated(subset=['siren', 'year', 'accountability', 'bundle'], keep='first')]

In [6]:
# On sélectionne uniquement les CR annuel complet, et on élimine certains doublons
bundle_raw = bundle_raw.loc[bundle_raw['accountability'] == 0,:].drop_duplicates(subset=['siren', 'year', 'accountability', 'bundle'], keep='first')
bundle_raw.head()

,siren,year,accountability,bundle,amount
0,5420120,2016,0,10,781843
1,5420120,2016,0,1,11836
2,5420120,2016,0,13,27644
3,5420120,2016,0,15,866430
4,5420120,2016,0,16,214907


In [7]:
# On charge le nom des colonnes
ontology = pd.read_json('http://api.enthic.fr/company/ontology')

# Création d'un dataframe contenant les colonnes de chaque type de bilan
df_col = pd.DataFrame(columns=[ontology.loc[i,:].iloc[0].get("1") for i in range(4)]+
                      ['liasse '+ ontology.loc[i,:].iloc[0].get("1") for i in range(4)],
                               index=range(68))

# Lecture des colonnes du compte de résultat complet
for type_bilan in range(4):
    dictionnaire = ontology.loc[type_bilan,:].iloc[0].get("code")
    nb_col = len(dictionnaire)
    
    for i in range(nb_col):
        df_col.iloc[i,type_bilan] = dictionnaire.get(str(i)).get("1")
        df_col.iloc[i,type_bilan+4] = dictionnaire.get(str(i)).get("0")
  
col_CR_complet = df_col.loc[:,['Compte annuel complet','liasse Compte annuel complet']]
col_CR_complet.to_csv(chemin + 'col_CR_complet.csv')
col_CR_complet.head(7)

,Compte annuel complet,liasse Compte annuel complet
0,"Autres impôts, taxes et versements assimilés",9Z
1,Ventes de marchandises,FA
2,Production vendue biens,FD
3,Production vendue services,FG
4,Chiffres d’affaires nets,FJ
5,Production stockée,FM
6,Production immobilisée,FN


In [8]:
# On transforme la structure des données
bundle = pd.DataFrame(columns = ['year'] + list(range(0, 68)))
for year in sorted(bundle_raw['year'].unique()):
    bundle_year = bundle_raw.loc[bundle_raw['year']==year,:].pivot(index='siren', columns='bundle', values='amount')
    bundle_year['year'] = year
    bundle = pd.concat([bundle,bundle_year])


bundle = bundle.astype(float)

bundle.columns = ['year'] + list(col_CR_complet['Compte annuel complet'])
bundle['siren'] = bundle.index
bundle.head(7)
# La nouvelle forme des données est [111013 rows x 68 columns] = 7M de celulles
# L'ancienne forme des données était [3000000 rows x 5 columns] = 15M de cellules

,year,"Autres impôts, taxes et versements assimilés",Ventes de marchandises,Production vendue biens,Production vendue services,Chiffres d’affaires nets,Production stockée,Production immobilisée,Subventions d’exploitation,"Reprises sur amortissements et provisions, transfert de charges",Autres produits,Total des produits d’exploitation,Achats de marchandises (y compris droits de douane),Variation de stock (marchandises),Achats de matières premières et autres approvisionnements,Variation de stock (matières premières et approvisionnements),Autres achats et charges externes,"Impôts, taxes et versements assimilés",Salaires et traitements,Charges sociales,Autres charges,Total des charges d’exploitation,Résultat d'exploitation,Bénéfice attribué ou perte transférée,Perte supportée ou bénéfice transféré,Produits financiers de participations,Produits des autres valeurs mobilières et créances de l’actif immobilisé,Autres intérêts et produits assimilés,Reprises sur provisions et transferts de charges financier,Différences positives de change,Produits nets sur cessions de valeurs mobilières de placement,Total des produits financiers,Dotations financières sur amortissements et provisions,Intérêts et charges assimilées,Différences négatives de change,Charges nettes sur cessions de valeurs mobilières de placement,Total des charges financières,Résultat financier,Résultat en cours avant impôts,Produits exceptionnels sur opérations de gestion,Produits exceptionnels sur opérations en capital,Reprises sur provisions et transferts de charges exceptionnel,Total des produits exceptionnels,Charges exceptionnelles sur opérations de gestion,Charges exceptionnelles sur opérations en capital,Dotations exceptionnelles aux amortissements et provisions,Total des charges exceptionnelles,Résultat exceptionnel,Participation des salariés aux résultats de l’entreprise,Impôts sur les bénéfices,Total des produits,Total des charges,Bénéfices ou perte (Total des produits ‐ Total des charges),Impôts différés (compte de résultat),Résultat net des sociétés mises en équivalence,Résultat net des entreprises intégrées,Résultat Groupe (Résultat net consolidé),Part des intérêts minoritaires (Résultat hors groupe),Résultat net part du groupe (part de la société mère),Rémunération d’intermédiaires et honoraires (hors rétrocessions),"Location, charges locatives et de copropriété",Effectif moyen du personnel,Sous‐traitance,Personnel extérieur à l’entreprise,"Rétrocessions d’honoraires, commissions et courtages",Taxe professionnelle,Montant de la TVA. collectée,Total TVA. déductible sur biens et services,Dividendes,siren
414381475,1919.0,NaN,2143339.0,189049.0,72029.0,2404417.0,NaN,NaN,NaN,6403.0,535.0,2411355.0,NaN,16200.0,4606.0,NaN,416966.0,14456.0,356608.0,101186.0,170.0,2240280.0,NaN,NaN,NaN,NaN,NaN,6911.0,NaN,NaN,NaN,6911.0,NaN,752.0,NaN,NaN,752.0,6158.0,NaN,472.0,NaN,NaN,NaN,NaN,1791.0,NaN,1791.0,-1319.0,NaN,52797.0,2418738.0,2295621.0,123117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414381475
420210577,2007.0,NaN,327650.0,NaN,19396.0,347046.0,NaN,NaN,NaN,9280.0,NaN,356326.0,176131.0,359.0,NaN,NaN,57107.0,4485.0,73776.0,28777.0,-98.0,360374.0,-4048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3338.0,NaN,NaN,3338.0,-3338.0,-7385.0,8434.0,25993.0,NaN,34427.0,318.0,NaN,NaN,26311.0,8116.0,NaN,599.0,390753.0,390621.0,132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420210577
527710172,2008.0,NaN,18849.0,NaN,847996.0,866844.0,NaN,NaN,NaN,5626.0,1505.0,873976.0,NaN,-8930.0,32.0,NaN,409266.0,46139.0,230059.0,55848.0,125363.0,957888.0,-83912.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12537.0,NaN,NaN,12537.0,-12537.0,-96449.0,NaN,NaN,NaN,2078.0,64.0,NaN,NaN,64.0,NaN,NaN,NaN,876054.0,970489.0,-94435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527710172
420210577,2009.0,NaN,268569.0,NaN,758.0,269327.0,NaN,NaN,NaN,4054.0,38.0,273419.0,132881.0,NaN,1115.0,NaN,48998.0,2568.0,50353.0,19658.0,39.0,265855.0,7564.0,NaN,NaN,NaN,Na

# 2. Chargement et préparation de ape, identity, ess

In [9]:
# Chargement et préparation de APE
ape = pd.read_json('http://api.enthic.fr/company/ape').transpose()
ape.columns = ['ape', 'ape_name']
ape['ape_len'] = ape['ape'].apply(lambda x: len(x))

print(ape['ape_len'].value_counts())
ape.head(7)

5    732
4    562
3    290
2     89
Name: ape_len, dtype: int64


,ape,ape_name,ape_len
0,1020Z,"Transform. & conserv. poisson, crustacés & mol...",5
1,812,Activités de nettoyage,3
2,1610B,Imprégnation du bois,5
3,0910Z,Activités de soutien à l'extraction d'hydrocar...,5
4,7420Z,Activités photographiques,5
5,4776Z,"Comm. dét. fleurs, plantes, etc, animaux de ci...",5
6,4742Z,Comm. détail matériels télécommunication (maga...,5


In [10]:
# Définition des catégories des ape
def get_cat(string, num):
  """ permet d'obtenir la catégorie de l'APE"""
  string = str(string)  # convertit les éventuels nombre en chaîne de caractère
  if num == 0:
    return string[0:2]
  elif len(string) > num+1:
    return string[0:num+2]
  else :
    return ''

ape['ape_division'] = ape['ape'].apply(lambda x: get_cat(x, 0))
ape['ape_groupe'] = ape['ape'].apply(lambda x: get_cat(x, 1))
ape['ape_classe'] = ape['ape'].apply(lambda x: get_cat(x, 2))
ape['ape_sous_classe'] = ape['ape'].apply(lambda x: get_cat(x, 3))

ape['enthic'] = ape.index
ape.head(7)

,ape,ape_name,ape_len,ape_division,ape_groupe,ape_classe,ape_sous_classe,enthic
0,1020Z,"Transform. & conserv. poisson, crustacés & mol...",5,10,102,1020,1020Z,0
1,812,Activités de nettoyage,3,81,812,,,1
2,1610B,Imprégnation du bois,5,16,161,1610,1610B,2
3,0910Z,Activités de soutien à l'extraction d'hydrocar...,5,09,091,0910,0910Z,3
4,7420Z,Activités photographiques,5,74,742,7420,7420Z,4
5,4776Z,"Comm. dét. fleurs, plantes, etc, animaux de ci...",5,47,477,4776,4776Z,5
6,4742Z,Comm. détail matériels télécommunication (maga...,5,47,474,4742,4742Z,6


In [11]:
# Chargement de identity
identity = pd.read_csv(chemin + 'identity.csv', sep=';', header=None,  index_col=False, names =['siren', 'denomination', 'enthic', 'postal_code','town'])

def numeriser(nombre):
  """ permet de numériser un nombre, si possible """
  try:
    num = int(nombre)
    return num
  except ValueError:
    return ''


identity['enthic'] = identity['enthic'].apply(numeriser)
identity.head(7)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,siren,denomination,enthic,postal_code,town
0,5420120,STE DES SUCRERIES DU MARQUENTERRE,362,62140,MARCONNELLE
1,5450093,SARL BAS ALPINE D AUTOCARS,273,04800,GREOUX LES BAINS
2,5450119,LE GRAND CAFE ET GASSENDI,1189,04000,DIGNE-LES-BAINS
3,5520176,HERNAS CARTONNAGES,1300,80210,FEUQUIERES-EN-VIMEU
4,5520242,PROSPA,1566,80510,LONGPRE-LES-CORPS-SAINTS
5,5541552,ETABLISSEMENTS BONNET FRERES,1457,04370,BEAUVEZER
6,5550108,EMILE BARNEAUD ET FILS,1003,04140,SEYNE


In [12]:
bundle['siren'].nunique()

618209

In [13]:
# Chargement de la liste des entreprises de l'ESS
df_ess = pd.read_csv(chemin + 'listeess_ldds_vjuin2020_ok_2.csv', sep=';', encoding='iso8859_16')
df_ess.columns = ['ess_' + x for x in df_ess.columns]
df_ess['is_ess'] = True
df_ess.head()

,ess_N° IDENTITE ENTREPRISE (SIREN),ess_RAISON SOCIALE,ess_ENSEIGNE,ess_SIGLE,ess_CP VILLE,ess_FORME JURIDIQUE,is_ess
0,5650148,SCIC VALDURANCE HABITAT,NaN,NaN,04000 DIGNE LES BAINS,Autres coopératives,True
1,5720610,PICARDIE MARITIME HABITAT FONDATION PAUL DUCLERCQ,NaN,NaN,80100 ABBEVILLE,Autres coopératives,True
2,5781133,IDEA GROUPE,NaN,MTTM,44550 MONTOIR DE BRETAGNE,Sociétés coopératives et participatives (SCOP),True
3,5880679,SA COOPERATIVE D'INTERET COLLECTIF POURL'ACCES...,NaN,NaN,44570 TRIGNAC,Autres coopératives,True
4,6080170,COOP BREIZH,NaN,NaN,29540 SPEZET,Autres coopératives,True


for encoding in list(df_encoding.loc[19:,'Codec']):
  try:
    df_ess = pd.read_csv(chemin + 'listeess_ldds_vjuin2020_ok_2.csv', nrows=100, sep=';', encoding=encoding, error_bad_lines=False)
    print('réussi :', encoding)
    pass
  except UnicodeDecodeError:
    pass
  except UnicodeDecodeError:
    pass
  else:
    pass
  finally:
    pass

# 3. Jointure de bundle, ape et identity

In [14]:
# Jointure de identity et de ape
identity_ape = identity.merge(ape, left_on='enthic', right_on='enthic', how='left')
identity_ape.head()

,siren,denomination,enthic,postal_code,town,ape,ape_name,ape_len,ape_division,ape_groupe,ape_classe,ape_sous_classe
0,5420120,STE DES SUCRERIES DU MARQUENTERRE,362,62140,MARCONNELLE,7010Z,Activités des sièges sociaux,5.0,70,701,7010,7010Z
1,5450093,SARL BAS ALPINE D AUTOCARS,273,04800,GREOUX LES BAINS,4778C,Autres commerces de détail spécialisés divers,5.0,47,477,4778,4778C
2,5450119,LE GRAND CAFE ET GASSENDI,1189,04000,DIGNE-LES-BAINS,5630Z,Débits de boissons,5.0,56,563,5630,5630Z
3,5520176,HERNAS CARTONNAGES,1300,80210,FEUQUIERES-EN-VIMEU,1721A,Fabrication de carton ondulé,5.0,17,172,1721,1721A
4,5520242,PROSPA,1566,80510,LONGPRE-LES-CORPS-SAINTS,2030Z,"Fabrication de peintures, vernis, encres et ma...",5.0,20,203,2030,2030Z


In [15]:
# Jointure de identity_ape et df_ess
identity_ape_ess = identity_ape.merge(df_ess, left_on='siren', right_on='ess_N° IDENTITE ENTREPRISE (SIREN)', how='left')
identity_ape_ess.head(5)

,siren,denomination,enthic,postal_code,town,ape,ape_name,ape_len,ape_division,ape_groupe,ape_classe,ape_sous_classe,ess_N° IDENTITE ENTREPRISE (SIREN),ess_RAISON SOCIALE,ess_ENSEIGNE,ess_SIGLE,ess_CP VILLE,ess_FORME JURIDIQUE,is_ess
0,5420120,STE DES SUCRERIES DU MARQUENTERRE,362,62140,MARCONNELLE,7010Z,Activités des sièges sociaux,5.0,70,701,7010,7010Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5450093,SARL BAS ALPINE D AUTOCARS,273,04800,GREOUX LES BAINS,4778C,Autres commerces de détail spécialisés divers,5.0,47,477,4778,4778C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5450119,LE GRAND CAFE ET GASSENDI,1189,04000,DIGNE-LES-BAINS,5630Z,Débits de boissons,5.0,56,563,5630,5630Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5520176,HERNAS CARTONNAGES,1300,80210,FEUQUIERES-EN-VIMEU,1721A,Fabrication de carton ondulé,5.0,17,172,1721,1721A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5520242,PROSPA,1566,80510,LONGPRE-LES-CORPS-SAINTS,2030Z,"Fabrication de peintures, vernis, encres et ma...",5.0,20,203,2030,2030Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Jointure de identity_ape et bundle
data = bundle.merge(identity_ape_ess, left_on='siren', right_on='siren', how='left')
data['is_ess'] = data['is_ess'].fillna(False)
data.shape

(1294936, 88)

In [19]:
data['is_ess'].value_counts()

False    1286674
True        8262
Name: is_ess, dtype: int64

# 4. Sauvegarde des données par catégorie

In [ ]:
#with open('/content/gdrive/My Drive/Impact Score/Export données par catégorie/data.xlsx', 'w') as f:
data.to_csv(chemin + 'all_data.csv')

In [ ]:
"""for j in ['0' + str(i) for i in range(0,10)]+ [str(i) for i in list(range(10,data['ape_cat0'].dropna().astype(int).max() +1))]:
  data[data['ape_cat0']==str(j)].to_excel('/content/gdrive/My Drive/Impact Score/Export données par catégorie/data{}.xlsx'.format(j))
  print(j)
  """